In [1]:
from integsol.mesh.mesh import Mesh
from integsol.structures.vectors import VectorField
from integsol.structures.operators import (
    IntegralConvolutionOperator as ICO,
    CrossProductOperator as CPO,
)
from integsol.validators.compute_validators import *
from integsol.structures.kernels import demagnetization_tensor_kernel as dtk
import numpy as np
from torch.linalg import eig
from torch import (
    Tensor,
    dot,
    matmul,
    mv,
)


In [2]:
M_s = 1.45 * 1e4
gamms = 2.25 * 1e4

In [ ]:
mesh = Mesh.read(path="/home/aluatar/integsol/test_inputs/mesh_75nm.mphtxt")
#mesh_2 = Mesh.read(path="/home/aluatar/integsol/test_inputs/mesh_coerser.mphtxt")

In [ ]:
centers = mesh.elements_centers['tet'].T
nodes = mesh.coordinates.T
elements = mesh.elements_coordinates
len(centers.T)

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt 
import numpy as np

fig = plt.figure()

ax = fig.add_subplot(projection='3d')

ax.scatter(xs=np.float64(nodes[0]), ys=np.float64(nodes[1]), zs=np.float64(nodes[2]), s=1)
#ax.scatter(xs=mesh.coordinates[Hd_nodes_nans][0], ys=mesh.coordinates[Hd_nodes_nans][1], zs=mesh.coordinates[Hd_nodes_nans][2], color='red')

In [176]:
coord_to_write = []
X, Y, Z =[], [], []
for coordinate in centers.T:
    X.append(coordinate[0])
    Y.append(coordinate[1])
    Z.append(coordinate[2])

coord_to_write = zip(X,Y,Z)

from datetime import datetime
import csv

with open(F"/home/aluatar/integsol/outputs/interpolation_coordinates/coordinates_{datetime.now()}.txt", 'w') as _csv:
    writer = csv.writer(_csv, delimiter='\t')
    writer.writerows(coord_to_write)

In [5]:
M0 = VectorField.read_to_mesh(
    path="/home/aluatar/integsol/test_inputs/magnetization_75nm.txt",
    mesh=mesh,
    dim=3)

In [ ]:
M0_cpo = CPO(
    mesh=mesh,
    left_vector= M0
)

M0_times_ = M0_cpo.to_mesh_matrix()
M0_times_
del M0

In [ ]:
ico = ICO(kernel=dtk)
int_G_ = ico.to_mesh_matrix(mesh=mesh)

In [ ]:
H_eff = VectorField.read_to_mesh(
    path="/home/aluatar/integsol/test_inputs/H_eff_75nm.txt",
    mesh=mesh,
    dim=3)

"""_H_eff = H_eff.vectorize()
vals_H_eff = H_eff.values.T"""

In [10]:
H_0 = VectorField(
    mesh=mesh,
    coordinates=centers.T,
    values= 5e6 * np.array([[0,0,1] for _ in centers.T])
)
H_0_coordinates = H_0.coorrdinates.T
H_0_values = H_0.values.T

In [ ]:
%matplotlib inline
%matplotlib widget 

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure(figsize=(5,5), dpi=100)
ax = fig.add_subplot(projection="3d") 

ax.quiver(
    H_0_coordinates[0],
    H_0_coordinates[1],
    H_0_coordinates[2],
    H_0_values[0],
    H_0_values[1],
    H_0_values[2],
    length=1e-5,
)

plt.show()

In [ ]:
H_0_cpo = CPO(
    mesh=mesh,
    left_vector=H_0
)
H_0_times_ = H_0_cpo.to_mesh_matrix()

In [ ]:
H_eff_cpo = CPO(
    mesh=mesh,
    left_vector=H_eff
)
H_eff_times_ = H_eff_cpo.to_mesh_matrix()
del H_eff

In [14]:
IntConvOp = matmul(M0_times_,int_G_)

In [ ]:
LLG_operator = gamms * (H_0_times_ + H_eff_times_  - M_s * IntConvOp)

del IntConvOp
LLG_operator

In [16]:
#omega_char = gamms * h_mx
#null_LLG = LLG_operator + 1j * omega_char * np.identity(n=len(LLG_operator))
#null_LLG#
#
#U, S, Vh = np.linalg.svd(null_LLG)

#null_space = np.compress(S <= 5.2e13, Vh, axis=0)
#null_space.T

dm_0 = VectorField(mesh=mesh, coordinates=centers.T, values=0.1 * np.array([[1 / np.sqrt(2), 1 / np.sqrt(2), 0] for _ in centers.T]))

dm_0_coordinates = dm_0.coorrdinates.T
dm_0_values = dm_0.values.T
_dm_0 = dm_0.vectorize()

In [ ]:
%matplotlib inline
%matplotlib widget 

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure(figsize=(5,5), dpi=100)
ax = fig.add_subplot(projection="3d") 

ax.quiver(
    dm_0_coordinates[0],
    dm_0_coordinates[1],
    dm_0_coordinates[2],
    dm_0_values[0],
    dm_0_values[1],
    dm_0_values[2],
    length=300,
)

plt.show()

In [ ]:
from scipy.sparse.linalg import eigs as speigs

eigs = speigs(np.array(LLG_operator), k=10, tol=1e-15)

In [ ]:
eigs

In [25]:
eigenvalues = eigs[0]#.eigenvalues
eigenvectors = eigs[1].T#.eigenvectors.T
eigenvalues = (1j * np.array(eigenvalues))
#eigenvalues_idx = np.where(abs(eigenvalues.imag) < 10000000)[0]
#eigenvalues = eigenvalues[eigenvalues_idx].real
#eigenvalues_idx = np.where(abs(eigenvalues) > 1e9)[0]
#eigenvalues = eigenvalues[eigenvalues_idx]
#eigenvalues, eigenvalues_idx

In [ ]:
eigenvalues[0]

In [ ]:
dm = VectorField(
    mesh=mesh,
    coordinates=mesh.elements_centers['tet'],
)
n = 3
_dm = eigenvectors[10] #+ eigenvectors[1] + eigenvectors[4] + eigenvectors[5] + eigenvectors[8] + eigenvectors[9]
dm.devectorize(np.array(_dm).real)

dm_values = dm.values.T
dm_coordinates = dm.coorrdinates.T
dm_values

In [ ]:
%matplotlib inline
%matplotlib widget 

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure(figsize=(10,10), dpi=50)
ax = fig.add_subplot(projection="3d") 

ax.quiver(
    dm_coordinates[0],
    dm_coordinates[1],
    dm_coordinates[2],
    dm_values[0],
    dm_values[1],
    dm_values[2],
    length=1000,
    
)

plt.show()

In [ ]:
eigenvalues[0]

In [ ]:
5e5 * gamms